# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

In [1]:
from langchain.agents import create_agent

In [4]:
agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[],
    prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

A SQL query walks into a bar, walks up to two tables and asks, "Can I join you?"

Want another one?


## values
You have seen this streaming mode in our examples so far. 

In [8]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

I'm reading a book on anti-gravity — it's impossible to put down.


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [9]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

Sunlight tiptoes through the blinds,
waking giggles, sleepy minds.
Pancakes flip like tiny moons,
while the kettle hums old tunes.

A cat in socks performs a jig,
and socks reply, "Don't be so big!"
The dog brings newspapers to the door,
then chases shadows on the floor.

We plant a seed, we tell a joke,
we count the clouds that look like smoke.
A kite takes off with a silly shout,
and teaches fear what it's about.

Hands held tight, we stroll along —
home is where our hearts belong.
At day’s end stars begin to peep;
we tuck our dreams in snug and deep.

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [10]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='d2d9cdc3-87e8-4753-956a-3a74fa3981e7')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='d2d9cdc3-87e8-4753-956a-3a74fa3981e7'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 132, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-COPssqbfQ0E89hs5E7PCdEzt9ZcP6', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--5d36a463-9ade-4e95-bef2-d3c89e89abf8-0', tool_calls=[{'name': 'get_wea

In [11]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: San Francisco
Acquired data for city: San Francisco


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])